In [1]:
import oasis
con = oasis.getTempCredentials()

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])
from adbnx_adapter.imdb_arangoDB_networkx_arango_adapter import IMDBArangoDB_Networkx_Adapter
ma = IMDBArangoDB_Networkx_Adapter(conn = con)

Reusing cached credentials.

https://5904e8d8a65f.arangodb.cloud:8529
Username: TUTcvzmdnahdqqtqg90jwhedl
Password: TUTy9qwp7aods0wa97zmyddl
Database: TUTlb2w846yqn2457bmox0lh


In [2]:
imdb_attributes = { 'vertexCollections': {'Users': {'user_id'},\
       'Movies': {'movie_id'}},\
                              'edgeCollections' : {'Ratings': {'_from', '_to', 'ratings'}}}

In [3]:
g = ma.create_networkx_graph(graph_name = 'IMDBGraph',  graph_attributes =   imdb_attributes)

In [4]:
g.nodes['Users/2']

{'attr_dict': {'user_id': '', '_id': 'Users/2'}, 'bipartite': 0}

In [5]:
g.nodes['Movies/4']

{'attr_dict': {'movie_id': '', '_id': 'Movies/4'}, 'bipartite': 1}

In [6]:
 m4v = [ t[0] for t in g.in_edges('Movies/4')]

In [7]:
from itertools import combinations
m4vucmb = list(combinations(m4v, 2))

In [8]:
m4v[1]

'Users/19'

In [9]:
import networkx as nx
gp = g.to_undirected()
jcp = nx.jaccard_coefficient(gp, m4vucmb)

In [10]:
gs = nx.DiGraph()
for u, v, p in jcp:
  gs.add_edge(u, v, weight = p)
  #print('(%s, %s) -> %.8f' % (u, v, p))

In [11]:
gs.number_of_edges()

9453

In [12]:
!pip install node2vec

In [13]:
from node2vec import Node2Vec
node2vec = Node2Vec(gs, dimensions=32, walk_length=10, num_walks=30, workers=4)

Computing transition probabilities: 100%|██████████| 138/138 [00:01<00:00, 127.62it/s]


In [14]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [15]:
model.wv.most_similar(m4v[5])

[('Users/92', 0.9994627237319946),
 ('Users/394', 0.9992709159851074),
 ('Users/393', 0.9992488622665405),
 ('Users/7', 0.9992043972015381),
 ('Users/621', 0.9991756677627563),
 ('Users/429', 0.9991444945335388),
 ('Users/535', 0.9990981817245483),
 ('Users/796', 0.9990875720977783),
 ('Users/151', 0.9990612268447876),
 ('Users/189', 0.9990416765213013)]